In [26]:
!pip install pyspark==3.0.0 koalas==1.0.1 pyarrow==0.15.1 scikit-learn==0.23.1

In [27]:
%%bash
for i in `seq 4`; do 
    rm log$i*
    wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log$i.txt
done

--2020-07-04 05:03:31--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19954 (19K) [text/plain]
Saving to: ‘log1.txt’

     0K .......... .........                                  100% 12.6M=0.002s

2020-07-04 05:03:31 (12.6 MB/s) - ‘log1.txt’ saved [19954/19954]

--2020-07-04 05:03:31--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13026 (13K) [text/plain]
Saving to: ‘log2.txt’

     0K .......... ..      

In [28]:
%%bash
for i in `seq 4`; do 
    echo "ts,rssi" > log$i.csv
    cat log$i.txt |awk  '{print $4","$11}' >> log$i.csv
done

In [29]:
%%bash
for i in `seq 4`; do 
    rm markers$i.csv
    wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers$i.csv
done

--2020-07-04 05:03:32--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers1.csv’

     0K                                                       100% 6.87M=0s

2020-07-04 05:03:32 (6.87 MB/s) - ‘markers1.csv’ saved [151/151]

--2020-07-04 05:03:32--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers2.csv’

     0K                          

In [30]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [31]:
from pyspark.sql.functions import lit

data = None
markers = None

for i in range(1,5):
    data_tmp = spark.read.option('header','true').csv('log'+str(i)+'.csv')
    data_tmp = data_tmp.withColumn('receiver_placement', lit(i))
    markers_tmp = spark.read.option('header','true').csv('markers'+str(i)+'.csv')
    markers_tmp = markers_tmp.withColumn('receiver_placement', lit(i))
    if i == 1:
        data = data_tmp
        markers = markers_tmp
    else:
        data = data.union(data_tmp)
        markers = markers.union(markers_tmp)

In [32]:
#convert ts_string to ts_timestamp
from pyspark.sql.functions import to_timestamp
data = data.withColumn("ts_p", to_timestamp("ts", "HH:mm:ss"))
markers = markers.withColumn("ts_start_p", to_timestamp("ts_start", "HH:mm:ss"))
markers = markers.withColumn("ts_stop_p", to_timestamp("ts_stop", "HH:mm:ss"))

In [33]:
data.createOrReplaceTempView('data')
markers.createOrReplaceTempView('markers')

In [34]:
data = spark.sql('''
select rssi,d.receiver_placement,location from data d 
inner join markers m on 
d.receiver_placement = m.receiver_placement 
and d.ts_p>=m.ts_start_p 
and d.ts_p<=m.ts_stop_p
''')
data.createOrReplaceTempView('data')

In [35]:
import databricks.koalas as ks
kdf = data.to_koalas()

In [36]:
kdf = kdf.dropna()

In [37]:
pdf = kdf.toPandas()

In [38]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(pdf[['rssi','receiver_placement']], pdf[['location']])

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(pdf[['location']], clf.predict(pdf[['rssi','receiver_placement']]))

0.27932960893854747

In [40]:
from sklearn.metrics import classification_report
target_names = ['0','1','2','3']
print(classification_report(pdf[['location']], clf.predict(pdf[['rssi','receiver_placement']]), target_names=target_names))

              precision    recall  f1-score   support

           0       0.36      0.67      0.47        92
           1       0.00      0.00      0.00        88
           2       0.00      0.00      0.00        81
           3       0.24      0.39      0.30        97

    accuracy                           0.28       358
   macro avg       0.15      0.27      0.19       358
weighted avg       0.16      0.28      0.20       358



/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
ks.DataFrame({},index=list(['rssi_1','rssi_2','rssi_3','rssi_4','location']))

""
rssi_1
rssi_2
rssi_3
rssi_4
location


In [42]:
data_pivot = spark.sql('''
    with data_1 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, location,rssi as rssi_1 from data where receiver_placement = 1
    ),

    data_2 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_2 from data where receiver_placement = 2
    ),

    data_3 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_3 from data where receiver_placement = 3
    ),

    data_4 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_4 from data where receiver_placement = 4
    )

    select * from (
        (select rssi_1, rssi_2, rssi_3, rssi_4, location from data_1 d1 inner join data_2 d2 on d1.id = d2.id inner join data_3 d3 on d2.id = d3.id inner join data_4 d4 on d3.id = d4.id)
    )
''')

In [56]:
data_l0 = spark.sql('select location, rssi, receiver_placement from data where location = 0')
data_l0.createOrReplaceTempView('data_l0')
data_l1 = spark.sql('select location, rssi, receiver_placement from data where location = 1')
data_l1.createOrReplaceTempView('data_l1')
data_l2 = spark.sql('select location, rssi, receiver_placement from data where location = 2')
data_l2.createOrReplaceTempView('data_l2')
data_l3 = spark.sql('select location, rssi, receiver_placement from data where location = 3')
data_l3.createOrReplaceTempView('data_l3')


data_pivot_0 = spark.sql('''
    with data_1 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, location,rssi as rssi_1 from data_l0 where receiver_placement = 1
    ),

    data_2 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_2 from data_l0 where receiver_placement = 2
    ),

    data_3 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_3 from data_l0 where receiver_placement = 3
    ),

    data_4 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_4 from data_l0 where receiver_placement = 4
    )

    select * from (
        (select rssi_1, rssi_2, rssi_3, rssi_4, location from data_1 d1 inner join data_2 d2 on d1.id = d2.id inner join data_3 d3 on d2.id = d3.id inner join data_4 d4 on d3.id = d4.id)
    )
''')

data_pivot_1 = spark.sql('''
    with data_1 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, location,rssi as rssi_1 from data_l1 where receiver_placement = 1
    ),

    data_2 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_2 from data_l1 where receiver_placement = 2
    ),

    data_3 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_3 from data_l1 where receiver_placement = 3
    ),

    data_4 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_4 from data_l1 where receiver_placement = 4
    )

    select * from (
        (select rssi_1, rssi_2, rssi_3, rssi_4, location from data_1 d1 inner join data_2 d2 on d1.id = d2.id inner join data_3 d3 on d2.id = d3.id inner join data_4 d4 on d3.id = d4.id)
    )
''')

data_pivot_2 = spark.sql('''
    with data_1 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, location,rssi as rssi_1 from data_l2 where receiver_placement = 1
    ),

    data_2 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_2 from data_l2 where receiver_placement = 2
    ),

    data_3 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_3 from data_l2 where receiver_placement = 3
    ),

    data_4 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_4 from data_l2 where receiver_placement = 4
    )

    select * from (
        (select rssi_1, rssi_2, rssi_3, rssi_4, location from data_1 d1 inner join data_2 d2 on d1.id = d2.id inner join data_3 d3 on d2.id = d3.id inner join data_4 d4 on d3.id = d4.id)
    )
''')

data_pivot_3 = spark.sql('''
    with data_1 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, location,rssi as rssi_1 from data_l3 where receiver_placement = 1
    ),

    data_2 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_2 from data_l3 where receiver_placement = 2
    ),

    data_3 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_3 from data_l3 where receiver_placement = 3
    ),

    data_4 as (
        select ROW_NUMBER() OVER (
            ORDER BY (SELECT NULL)
        ) as ID, rssi as rssi_4 from data_l3 where receiver_placement = 4
    )

    select * from (
        (select rssi_1, rssi_2, rssi_3, rssi_4, location from data_1 d1 inner join data_2 d2 on d1.id = d2.id inner join data_3 d3 on d2.id = d3.id inner join data_4 d4 on d3.id = d4.id)
    )
''')

In [60]:
data_pivot = data_pivot_0.union(data_pivot_1).union(data_pivot_2).union(data_pivot_3)

In [61]:
data_pivot_pandas = data_pivot.toPandas()

,rssi_1,rssi_2,rssi_3,rssi_4,location
0,-86,-84,-97,-99,0
1,-84,-86,-99,-98,0
2,-86,-91,-104,None,0
3,-87,-90,-100,-99,0
4,-77,-86,-103,None,0
5,-72,-88,None,-100,0
6,-66,-87,None,-99,0
7,-73,-88,-97,-99,0
8,-67,-91,None,-100,0
9,-74,-87,None,None,0


In [64]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(data_pivot_pandas[['rssi_1','rssi_2','rssi_3','rssi_4',]], data_pivot_pandas[['location']])

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(data_pivot_pandas[['location']], clf.predict(data_pivot_pandas[['rssi_1','rssi_2','rssi_3','rssi_4',]]))

1.0

In [66]:
from sklearn.metrics import classification_report
target_names = ['0','1','2','3']
print(classification_report(data_pivot_pandas[['location']], clf.predict(data_pivot_pandas[['rssi_1','rssi_2','rssi_3','rssi_4',]]), target_names=target_names))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00        19

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42



In [63]:
data_pivot_pandas = data_pivot_pandas.dropna()
data_pivot_pandas

,rssi_1,rssi_2,rssi_3,rssi_4,location
0,-86,-84,-97,-99,0
1,-84,-86,-99,-98,0
3,-87,-90,-100,-99,0
7,-73,-88,-97,-99,0
12,-62,-85,-98,-102,0
13,-76,-87,-102,-99,0
19,-90,-73,-95,-97,1
20,-90,-55,-85,-103,1
21,-100,-72,-89,-98,1
22,-89,-72,-89,-98,1


In [ ]:
data_pivot.select('location').distinct().show()

In [ ]:
slices = [[x]*4 for x in [b for b in range(4)]]

for receiver_placement in range(1,5):
    for location in range(4):
        slices[location][receiver_placement-1] = kdf[(kdf['receiver_placement']==receiver_placement) & (kdf['location']==location)][['rssi','location']]


In [ ]:
minimum = 100
for i in range(4):
    for j in range(4):
        minimum = min(minimum,len(slices[i][j]))

In [ ]:
kdf_final = ks.range(minimum*4)
for i in range(4): #locations
    for j in range(4): #placements
        slices[i][j] = slices[i][j].head(minimum)


In [ ]:
ks.options.compute.ops_on_diff_frames=True
import pandas as pd
for i in range(4): #locations
    kdf_tmp = ks.range(minimum)
    for j in range(4): #placements
        if j == 0:
             kdf_tmp.join(pd.DataFrame(slices[i][j]['location']))
        kdf_tmp['rssi_'+str(j)]=slices[i][j]['rssi']
        print(slices[i][j]['rssi'])  